# Project summary

To be filled out later

# Data import and cleaning

## Importing libraries and seperating DataFrames

We begin be importing the libraries.

In [196]:
#Importing external libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Importing classes and functions from external files:
from model_classes import ModelSwitcher, DataPreprocessor
import cleaning_functions

#Configuring options
%matplotlib inline
pd.set_option('display.max_columns', 50)

In [197]:
df_outcomes = pd.read_csv("DATASETS/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv")
test_set = pd.read_csv("DATASETS/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv")
df = pd.read_csv("DATASETS/4910797b-ee55-40a7-8668-10efd5c1b960.csv")

Merging the test set with the set for evaluation before data cleaning optimizes the process by having the full scope of the values from the start.

In [198]:
df_toclean = pd.concat([df,test_set], axis=0)

Created a seperate dataframe for EDA which combines the training set and their respective values.

In [199]:
df_j = pd.merge(df, df_outcomes, how="left", on="id")

In [200]:
def dummy_creation():
    pass

def extract_impute_values(df, target, column, bad_data):
    isolate_good = df[df[column] != bad_data]
    values = isolate_good.groupby(target).mean()[column]
    return values.to_dict()

def imputer(df, target, column, val_dict, bad_data):
    for key, value in val_dict.items():
        df[column] = np.where((df[target].values == key) & (df[column].values == bad_data), value, df[column])
    return df

def simp_datetime_map(df, col, format=None):
    df[col] = df[col].map(lambda x: pd.to_datetime(x, format=format))
    return df
    
def main_data_cleaning(df):
    df = simp_datetime_map(df, "date_recorded")
    df = imputer(df_j, target_col, "construction_year", year_dict, 0)
    return df

I captured the information from the target value to better approximate missing values sending the results to dictionaries

In [ ]:
year_dict = extract_impute_values(df_j, target_col, "construction_year", 0)

In [201]:
df_c = main_data_cleaning(df_toclean)

In [202]:
df_j["construction_year"].unique() == df_c["construction_year"].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [203]:
df_j["region_code"].unique()

array([11, 20, 21, 90, 18,  4, 17, 14, 60, 10,  3, 15, 19, 16, 80,  1,  6,
        2, 12, 13,  5,  7, 99, 24,  9,  8, 40])

In [193]:
df_j[df_j["region_code"] == 10]["region"].unique()
# df["region"].unique().shape
# df["region_code"].unique().shape


array(['Ruvuma'], dtype=object)

array([1999.        , 2010.        , 2009.        , 1986.        ,
       1999.9386749 , 1992.39753787, 2011.        , 1987.        ,
       1991.        , 1978.        , 1992.        , 2008.        ,
       1974.        , 2000.        , 2002.        , 2004.        ,
       1972.        , 2003.        , 1980.        , 2007.        ,
       1973.        , 1995.25791139, 1985.        , 1970.        ,
       1995.        , 2006.        , 1962.        , 2005.        ,
       1997.        , 2012.        , 1996.        , 1977.        ,
       1983.        , 1984.        , 1990.        , 1982.        ,
       1976.        , 1988.        , 1989.        , 1975.        ,
       1960.        , 1961.        , 1998.        , 1963.        ,
       1971.        , 1994.        , 1968.        , 1993.        ,
       2001.        , 1979.        , 1967.        , 2013.        ,
       1969.        , 1981.        , 1964.        , 1966.        ,
       1965.        ])

In [159]:
df["amount_tsh"].unique()

array([6.00e+03, 0.00e+00, 2.50e+01, 2.00e+01, 2.00e+02, 5.00e+02,
       5.00e+01, 4.00e+03, 1.50e+03, 6.00e+00, 2.50e+02, 1.00e+01,
       1.00e+03, 1.00e+02, 3.00e+01, 2.00e+03, 4.00e+02, 1.20e+03,
       4.00e+01, 3.00e+02, 2.50e+04, 7.50e+02, 5.00e+03, 6.00e+02,
       7.20e+03, 2.40e+03, 5.00e+00, 3.60e+03, 4.50e+02, 4.00e+04,
       1.20e+04, 3.00e+03, 7.00e+00, 2.00e+04, 2.80e+03, 2.20e+03,
       7.00e+01, 5.50e+03, 1.00e+04, 2.50e+03, 6.50e+03, 5.50e+02,
       3.30e+01, 8.00e+03, 4.70e+03, 7.00e+03, 1.40e+04, 1.30e+03,
       1.00e+05, 7.00e+02, 1.00e+00, 6.00e+01, 3.50e+02, 2.00e-01,
       3.50e+01, 3.06e+02, 8.50e+03, 1.17e+05, 3.50e+03, 5.20e+02,
       1.50e+01, 6.30e+03, 9.00e+03, 1.50e+02, 1.20e+05, 1.38e+05,
       3.50e+05, 4.50e+03, 1.30e+04, 4.50e+04, 2.00e+00, 1.50e+04,
       1.10e+04, 5.00e+04, 7.50e+03, 1.63e+04, 8.00e+02, 1.60e+04,
       3.00e+04, 5.30e+01, 5.40e+03, 7.00e+04, 2.50e+05, 2.00e+05,
       2.60e+04, 1.80e+04, 2.60e+01, 5.90e+02, 9.00e+02, 9.00e

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

# EDA

In [26]:
cat_cols = ["quality_group"]
df_j.quality_group.unique()
sns.distplot(target_col, labels=cat_cols)

TypeError: distplot() got an unexpected keyword argument 'labels'

In [10]:
target_col = "status_group"
cat_cols = ["date_recorded", "funder", "installer", "wpt_name"]
cont_features = df_j.columns.drop(labels=[target_col, *cat_cols])
cont_features = ["gps_height", "longitude"]

In [11]:
data = DataPreprocessor(df_j, target_col, cont_features=cont_features, scale_type="minmax", poly_degree=2)

Getting polynomial features of degree 2
Skipping class imbalance functions
Using Min/Max scaler


In [12]:
model = ModelSwitcher(data, duplicate=True)

In [13]:
model.data.poly_degree = 1
model.data._poly_features()
model.data.X

Getting polynomial features of degree 1


,gps_height,longitude
0,1390,34.938093
1,1399,34.698766
2,686,37.460664
3,263,38.486161
4,0,31.130847
...,...,...
59395,1210,37.169807
59396,1212,35.249991
59397,0,34.017087
59398,0,35.861315
